In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import image
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
label_binarizer = LabelBinarizer()


import cv2
import pickle
img_data=[]
labels=[]

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from keras.preprocessing.image import ImageDataGenerator
#print(os.listdir('/kaggle/input'))
label_list=os.listdir('/kaggle/input/plantdisease/plantvillage/PlantVillage')
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
print(n_classes)
img_data=[]
labels_data=[]
# Any results you write to the current directory are saved as output.

ModuleNotFoundError: No module named 'keras'

In [16]:
from keras.models import model_from_json

In [3]:
train_gen=ImageDataGenerator(rescale=1.0/255,rotation_range=40,width_shift_range=0.2,shear_range=0.2,zoom_range=0.3,horizontal_flip=True,fill_mode="nearest")
train_generator=train_gen.flow_from_directory("/kaggle/input/plantdisease/plantvillage/PlantVillage",target_size=(224,224),batch_size=64,class_mode='categorical')

Found 20638 images belonging to 15 classes.


In [4]:
x,y=train_generator.next()
print(x.shape)
print(y.shape)

(64, 224, 224, 3)
(64, 15)


In [5]:
from keras.applications.vgg16 import VGG16
from keras.optimizers import adam
from keras.layers import *
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import Adam

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [7]:
opt=Adam(lr=1e-3)

In [8]:
model=Sequential()
model.add(Conv2D(32, (3, 3), padding="same",input_shape=(224,224,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)       

In [11]:
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['acc'])

In [12]:
hist=model.fit_generator(train_generator,epochs=7,steps_per_epoch=322,validation_data=(x_test,y_test))

Epoch 1/7
322/322 [==============================] - 279s 867ms/step - loss: 2.2390 - acc: 0.2648 - val_loss: 2.2079 - val_acc: 0.3500
Epoch 2/7
322/322 [==============================] - 240s 744ms/step - loss: 1.4884 - acc: 0.5063 - val_loss: 1.3198 - val_acc: 0.6500
Epoch 3/7
322/322 [==============================] - 239s 743ms/step - loss: 0.9889 - acc: 0.6704 - val_loss: 0.9476 - val_acc: 0.6000
Epoch 4/7
322/322 [==============================] - 241s 748ms/step - loss: 0.7457 - acc: 0.7491 - val_loss: 0.6287 - val_acc: 0.7500
Epoch 5/7
322/322 [==============================] - 240s 744ms/step - loss: 0.6048 - acc: 0.7965 - val_loss: 0.4176 - val_acc: 0.9000
Epoch 6/7
322/322 [==============================] - 239s 742ms/step - loss: 0.5175 - acc: 0.8272 - val_loss: 0.2312 - val_acc: 0.9500
Epoch 7/7
322/322 [==============================] - 239s 742ms/step - loss: 0.4502 - acc: 0.8511 - val_loss: 0.2490 - val_acc: 0.9000


In [19]:
model.save_weights("model.h5")
model_json=model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")    

In [25]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(x_test,y_test,verbose=`0)

Loaded model from disk


In [26]:
print(score)

[0.2490125149488449, 0.8999999761581421]


In [ ]:
predictions=model.predict(x_test)
scores=model.evaluate(x_test,y_test)
print(scores[1]*100)
#labels=np.argmax(predictions,axis=-1)
#print(labels)